In [4]:
from flask import Flask, render_template, request, jsonify
import joblib  # or any other library to load your machine learning model
import pickle
import pandas as pd
import numpy as np
from flask_cors import CORS

path = 'clean_data.csv'
data = pd.read_csv(path)
data['PCOS_label'] = None

data = data.set_index('PCOS_label')
data = data.reset_index()

def label(row):
    if row['PCOS'] == 'Yes':
        return 1
    else:
        return 0
    
data['PCOS_label'] = data.apply(lambda row: label(row), axis=1)

PCOS_check = dict(zip(data.PCOS_label.unique(), data.PCOS.unique()))
PCOS_check

filename = 'model.pkl'
loaded_model = pickle.load(open(filename, 'rb'))


feature_cols = ['Period Length', 'Cycle Length', 'Age', 'Overweight', 'loss weight gain / weight loss', 'irregular or missed periods', 'Difficulty in conceiving', 'Hair growth on Chin', 'Hair growth  on Cheeks', 'Hair growth Between breasts',
                'Hair growth  on Upper lips ', 'Hair growth in Arms', 'Hair growth on Inner thighs', 'Acne or skin tags', 'Hair thinning or hair loss ', 'Dark patches', 'always tired', 'more Mood Swings', 'exercise per week', 'eat outside per week', 'canned food often']

app = Flask(__name__)
CORS(app)
# Load your machine learning model
# Replace 'your_model_file.pkl' with the actual filename
model = joblib.load('model.pkl')


@app.route('/predict', methods=['POST'])
def predict():
    data=request.get_json()
    x_test = data['features']
    received_array = np.array(x_test).reshape(1, -1)
    result = loaded_model.predict(received_array)
    result=PCOS_check[result[0]]

    print(received_array)
    return jsonify({'result':result})

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


[2024-02-12 09:08:24,660] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "/home/animesh/miniconda3/envs/myenv2/lib/python3.10/site-packages/flask/app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/animesh/miniconda3/envs/myenv2/lib/python3.10/site-packages/flask/app.py", line 872, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/animesh/miniconda3/envs/myenv2/lib/python3.10/site-packages/flask_cors/extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/home/animesh/miniconda3/envs/myenv2/lib/python3.10/site-packages/flask/app.py", line 870, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/animesh/miniconda3/envs/myenv2/lib/python3.10/site-packages/flask/app.py", line 855, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-re

[[5 1 2 0 0 0 0 1 0 0 1 1 0 1 0 1 0 0 3 3 0]]


127.0.0.1 - - [12/Feb/2024 09:10:21] "OPTIONS /predict HTTP/1.1" 200 -
/home/animesh/miniconda3/envs/myenv2/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [12/Feb/2024 09:10:22] "POST /predict HTTP/1.1" 200 -


[[ 9  1  4  1  1  1  1  0  1  1  1  1  1  1  1  1  1  1  4 44  4]]
